In [1]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
import nltk
import re
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances

In [2]:
#Import the resume from PDF to strings. Change the pdf file name to test it.

from tika import parser # pip install tika

resumes=['dummy','Zehai (Steve)Wang','MILI SHAH','Jay Bakshi','Aman','Raman','PM_Resume']


raw_jd = parser.from_file('JD.pdf')
jd=raw_jd['content']

raw_resume_ds1 = parser.from_file('Zehai (Steve)Wang.pdf')
resume1=raw_resume_ds1['content']

raw_resume_ds2 = parser.from_file('MILI SHAH.pdf')
resume2=raw_resume_ds2['content']

raw_resume_ds3 = parser.from_file('Jay Bakshi.pdf')
resume3=raw_resume_ds3['content']

raw_resume_ds4 = parser.from_file('Aman.pdf')
resume4=raw_resume_ds4['content']

raw_resume_ds4 = parser.from_file('Raman.pdf')
resume5=raw_resume_ds4['content']

raw_resume_PM = parser.from_file('PM.pdf')
resume_PM=raw_resume_PM['content']

In [3]:
documents=[jd,resume1,resume2,resume3,resume4,resume5,resume_PM]

In [4]:
pd.set_option('display.max_colwidth', 0)
pd.set_option('display.max_columns', 0)

In [5]:
documents_df=pd.DataFrame(documents,columns=['documents'])

### Corpus Dataframe

In [6]:
# removing special characters and stop words from the text
stop_words_l=stopwords.words('english')
documents_df['documents_cleaned']=documents_df.documents.apply(lambda x: " ".join(re.sub(r'[^a-zA-Z]',' ',w).lower() for w in x.split() if re.sub(r'[^a-zA-Z]',' ',w).lower() not in stop_words_l) )


#### Tf-idf vectors

In [7]:
tfidfvectoriser=TfidfVectorizer(max_features=10)
tfidfvectoriser.fit(documents_df.documents_cleaned)
tfidf_vectors=tfidfvectoriser.transform(documents_df.documents_cleaned)

In [8]:
tfidf_vectors

<7x10 sparse matrix of type '<class 'numpy.float64'>'
	with 49 stored elements in Compressed Sparse Row format>

In [9]:
tfidf_vectors.shape

(7, 10)

In [10]:
# Every vector is already normalised to have unit L2 norm
# np.linalg.norm(tfidf_vectors[0],ord=2)

In [11]:
tfidf_vectors=tfidf_vectors.toarray()
print (tfidf_vectors[0])

[0.         0.         0.98766449 0.15022697 0.         0.
 0.         0.         0.04416684 0.        ]


In [12]:
df_test=pd.DataFrame(tfidf_vectors)

In [13]:
pairwise_similarities=np.dot(tfidf_vectors,tfidf_vectors.T)
pairwise_differences=euclidean_distances(tfidf_vectors)

In [14]:
pairwise_similarities

array([[1.        , 0.78445161, 0.3153716 , 0.76956347, 0.40144105,
        0.36383178, 0.06762463],
       [0.78445161, 1.        , 0.68687421, 0.87307498, 0.62706571,
        0.586008  , 0.28142564],
       [0.3153716 , 0.68687421, 1.        , 0.78907769, 0.44723207,
        0.42064903, 0.44448434],
       [0.76956347, 0.87307498, 0.78907769, 1.        , 0.63945006,
        0.60499349, 0.34219945],
       [0.40144105, 0.62706571, 0.44723207, 0.63945006, 1.        ,
        0.99774927, 0.19149623],
       [0.36383178, 0.586008  , 0.42064903, 0.60499349, 0.99774927,
        1.        , 0.18316635],
       [0.06762463, 0.28142564, 0.44448434, 0.34219945, 0.19149623,
        0.18316635, 1.        ]])

In [15]:
print (tfidf_vectors[0])
print (pairwise_similarities.shape)
print (pairwise_similarities[0][:])

[0.         0.         0.98766449 0.15022697 0.         0.
 0.         0.         0.04416684 0.        ]
(7, 7)
[1.         0.78445161 0.3153716  0.76956347 0.40144105 0.36383178
 0.06762463]


In [16]:
output=pairwise_similarities[0][:]


In [20]:
final_output=pd.DataFrame(output,resumes,columns=['Resume Score'])

In [22]:
final_output.drop(final_output.head(1).index, inplace=True)

In [26]:
final_output.sort_values('Resume Score',ascending=False)

,Resume Score
Zehai (Steve)Wang,0.784452
Jay Bakshi,0.769563
Aman,0.401441
Raman,0.363832
MILI SHAH,0.315372
PM_Resume,0.067625
